<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/gabs-short-intro-to-vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - QuickStart

## Vector Searches & Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.

Este material demonstra rapidamente como você pode usar o Redis como um VectorDB. Existe um hands-on mais completo, onde eu explico cada passo exatamente.

Quando falamos que existe um mundo de diferenças entre **SEARCH X QUERY**, a história começa aqui.

O Redis é o motor para buscas fonéticas, complexas, por sinonimo, por wildcard, por geolocalização, por ranges... e também por Vetor! Qualquer modelo que extraia a semântica por trás dos inputs é bem vindo aqui!

Espero que gostem!

# Objetivo do Laboratório

Lembre-se: Existe uma versão deste mesmo lab onde eu explico passo a passo o que estamos fazendo. Não se preocupe se você não entendeu de primeira. O objetivo aqui é de DEMO.

Basicamente, isso que faremos aqui:

*   Guardar uma base de dados de Bicicletas (como SKUs) no Redis.
*   Usar um modelo de Sentence Transformers da HuggingFace para gerar os embeddings a partir da descrição em linguagem natural da bicicleta.
*   Abrir um prompt de busca para o usuário final, onde iremos vetorizar a sua própria pergunta (input) usando o mesmo modelo de ML.
*   Efetuar uma busca de VSS/KNN contra o Redis, usando os embeddings (vectors) para encontrar a melhor resposta para a pergunta do usuário.



# Pré Requisito - Criar uma conta Free no Redis Cloud

Basta seguir o passo a passo [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-notebook-validator/blob/master/redis-workshop-setup-notebook-validator.ipynb)!

# Vector Similarity Search em 3 Passos!


## Passo 1 - Instalação e Configuração

In [5]:
# Instale as dependências
!pip install -r https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/requirements.txt
!apt-get update
!apt-get install -y redis-tools

# Conecte ao Redis
import os
import redis

REDIS_HOST="redis-18884.c98.us-east-1-4.ec2.redns.redis-cloud.com"
REDIS_PORT=18884
REDIS_PASSWORD="lgZgS90vZJpnS4F2Y5EJ97YJTFGUUdvF"

redis_conn = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

assert redis_conn.ping()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
redis-tools is already the newest version (5:6.0.16-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


## Passo 2 - Carregar as Bikes e indexá-las no Redis

In [4]:
import requests

# Carregar dados das bikes
URL = "https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/data/bikes.json"
bikes = requests.get(URL).json()
index_name = "idx:bikes_vss"

# Carregar dados no Redis
pipeline = redis_conn.pipeline()
for i, bike in enumerate(bikes, start=1):
    redis_key = f"bikes:{i:03}"
    pipeline.json().set(redis_key, "$", bike)
pipeline.execute()


# Function para checar se o index ja existe
def index_exists(index_name):
    try:
        # Vai lancar um error se o index nao existir
        redis.ft(index_name).info()
        return True
    except ResponseError:
        return False

# usa a function pra garantir uma exp atomica
if index_exists(index_name):
    print("Deleting older index version...")
    redis.execute_command("FT.DROPINDEX", index_name)


# Criar índice para busca vetorial
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

VECTOR_DIMENSION = 384

schema = [
    TextField("$.model", as_name="model"),
    TextField("$.brand", as_name="brand"),
    NumericField("$.price", as_name="price"),
    VectorField("$.description_embeddings", "FLAT", {"TYPE": "FLOAT32", "DIM": VECTOR_DIMENSION, "DISTANCE_METRIC": "COSINE"}, as_name="vector")
]

index_def = IndexDefinition(prefix=["bikes:"], index_type=IndexType.JSON)
redis_conn.ft(index_name).create_index(fields=schema, definition=index_def)


NameError: name 'ResponseError' is not defined

In [ ]:
# Testando via Python (redis-py)
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

## 1 - Importando e preparando as libs que iremos usar

Este primeiro bloco vai garantir que todas as dependências estejam prontas pra gente brincar com o lab.

In [ ]:
import json
import time

import numpy as np
import pandas as pd
import requests
import redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)


redis.ping()

True


## 2 - Carregando a massa de dados - Bikes

Vamos ingerir uma pequena massa de dados que contém bikes e suas descrições... como um SKU da vida.





In [ ]:
URL = "https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/data/bikes.json"
response = requests.get(URL, timeout=10)
bikes = response.json()

# vamos ver o que foi carregado
json.dumps(bikes[0], indent=2)

'{\n  "model": "Jigger",\n  "brand": "Velorim",\n  "price": 270,\n  "type": "Bicicletas Infantis",\n  "specs": {\n    "material": "alum\\u00ednio",\n    "weight": "10"\n  },\n  "description": "Pequena e poderosa, a Jigger \\u00e9 a melhor pedalada para os menores ciclistas! Esta \\u00e9 a menor bicicleta infantil com pedal no mercado dispon\\u00edvel sem freio de contrapedal, a Jigger \\u00e9 a escolha ideal para o raro pequeno tenaz pronto para partir. Dizemos raro porque esta pequena bicicleta n\\u00e3o \\u00e9 ideal para um iniciante nervoso, mas \\u00e9 um verdadeiro galope para um verdadeiro velocista. A Jigger \\u00e9 uma bicicleta infantil leve de 12 polegadas e atender\\u00e1 \\u00e0 necessidade de velocidade do seu pequeno. \\u00c9 uma bicicleta de uma \\u00fanica velocidade que torna o aprendizado a pedalar simples e intuitivo. Ela at\\u00e9 tem uma al\\u00e7a na parte inferior do selim para que voc\\u00ea possa ajudar seu filho facilmente durante o treinamento! A Jigger est\

## 3 - Carregar os dados no Redis como JSON - Binary Tree (não é uma String)

Vamos carregar essas bikes como documentos no Redis. Documentos JSON, claro!

Vamos usar o conceito de pipeline, que pode ser muito útil com volumes maiores de dados. O Redis Cloud tem um proxy zero-latency aqui pra cuidar do multiplexing.

In [ ]:
pipeline = redis.pipeline()
for i, bike in enumerate(bikes, start=1):
    redis_key = f"bikes:{i:03}"
    pipeline.json().set(redis_key, "$", bike)
res = pipeline.execute()

Com os dados carregados, podemos pegar trechos do documento JSON dessa maneira:

In [ ]:
res = redis.json().get("bikes:010", "$.model")
print(res)

['Summit']


## 4 - Escolha um modelo de embedding que entenda o Português Brasileiro

A **HuggingFace** possui um extenso catálogo de modelos de **embedding** de texto que podem ser servidos localmente através do framework **SentenceTransformers**.

Os gringos costumam usar o modelo MS MARCO, amplamente utilizado em mecanismos de busca, chatbots e outras aplicações de IA.

Entretanto, quero que este lab funcione com o nosso lindo idioma. Vamos fazer com `paraphrase-multilingual-MiniLM-L12-v2`

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

## 5 - Hora do Show: Gerando os embeddings no Redis!

O primeiro passo aqui é iterar e selecionar as chaves que vamos trabalhar. Neste caso, são as `bikes::`

In [ ]:
keys = sorted(redis.keys("bikes:*"))

Agora, use as chaves como entrada para o comando `JSON.MGET`, juntamente com o campo `$.description`, para coletar as descrições em uma lista.

Em seguida, passe a lista de descrições para o método `.encode()`:

In [ ]:
descriptions = redis.json().mget(keys, "$.description")

descriptions = [item for sublist in descriptions for item in sublist]

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

embeddings = embedder.encode(descriptions).astype(np.float32).tolist()

VECTOR_DIMENSION = len(embeddings[0])

print("Gabs says: This ST creates embeddings with {0} dimensions. Redis MUST know that. :D".format(VECTOR_DIMENSION))

Gabs says: This ST creates embeddings with 384 dimensions. Redis MUST know that. :D


Finalmente, insira as descrições vetorizadas nos documentos de bicicletas no Redis usando o comando `JSON.SET`.

O seguinte comando insere um novo campo em cada um dos documentos sob o **JSONPath** `$.description_embeddings`.

Mais uma vez, faça isso usando um **pipeline** para evitar viagens desnecessárias pela rede:

In [ ]:
#print(keys/embeddings)

pipeline = redis.pipeline()
for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, "$.description_embeddings", embedding)
pipeline.execute()

[True, True, True, True, True, True, True, True, True, True, True]

Vai ficar meio poluído aqui... mas olhem só como o dado está no Redis.

Podem usar o RedisInsight também!

In [ ]:
import json

res = redis.json().get("bikes:010")
pretty_res = json.dumps(res, indent=4, ensure_ascii=False)
print(pretty_res)


{
    "model": "Summit",
    "brand": "nHill",
    "price": 1200,
    "type": "Bicicleta de Montanha",
    "specs": {
        "material": "liga",
        "weight": "11.3"
    },
    "description": "Esta bicicleta de montanha econômica da nHill tem um bom desempenho tanto em ciclovias quanto em trilhas. O garfo com 100mm de curso absorve terrenos acidentados. Pneus largos Kenda Booster oferecem aderência em curvas e trilhas molhadas. A transmissão Shimano Tourney oferece engrenagens suficientes para encontrar um ritmo confortável para subir, e os freios a disco hidráulicos Tektro freiam suavemente. Se você quer uma bicicleta acessível que pode levar para o trabalho, mas também levar para trilhas nos fins de semana, ou está apenas atrás de uma pedalada estável e confortável na ciclovia, a Summit oferece um bom custo-benefício.",
    "description_embeddings": [
        0.028230920433998108,
        0.1364552080631256,
        -0.06720077246427536,
        -0.009281334467232227,
        -0

## 6 - Index e FT - Habilitando o Redis Query Engine nos dados

Agora, devemos criar um **INDEX** para consultar metadados de documentos ou realizar buscas vetoriais também. Use o comando `FT.CREATE`.

Aqui vai uma descrição mais completa do que estamos fazendo pro Redis entender o embedding como um vector array que pode ser usado nas consultas.

Você pode encontrar mais detalhes sobre todas essas opções na documentação de referência de vetores.


- **$.description_embeddings AS vector:** O caminho JSON do campo vetorial e seu alias de campo vector.
- **FLAT:** Especifica o método de indexação, que pode ser um índice plano (flat index) ou um gráfico hierárquico navegável pequeno mundo (HNSW).
- **TYPE FLOAT32:** Define a precisão de ponto flutuante de um componente do vetor, neste caso, um número de ponto flutuante de 32 bits.
- **DIM 384:** O comprimento ou dimensão dos embeddings, determinado pelo modelo de incorporação escolhido.
- **DISTANCE_METRIC COSINE:** A função de distância escolhida: distância cosseno.


In [ ]:
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.exceptions import ResponseError

# Function to check if the index exists
def index_exists(index_name):
    try:
        # This will throw an error if the index does not exist
        redis.ft(index_name).info()
        return True
    except ResponseError:
        return False

index_name = "idx:bikes_vss"

# Check if the index exists and drop it if it does
if index_exists(index_name):
    print("Deleting older index version...")
    redis.execute_command("FT.DROPINDEX", index_name)

# Define the schema
schema = (
    TextField("$.model", no_stem=True, as_name="model"),
    TextField("$.brand", no_stem=True, as_name="brand"),
    NumericField("$.price", as_name="price"),
    TagField("$.type", as_name="type"),
    TextField("$.description", as_name="description"),
    VectorField(
        "$.description_embeddings",
        "FLAT",
        {
            "TYPE": "FLOAT32",
            "DIM": VECTOR_DIMENSION,
            "DISTANCE_METRIC": "COSINE",
        },
        as_name="vector",
    ),
)

# Define the index definition
definition = IndexDefinition(prefix=["bikes:"], index_type=IndexType.JSON)

# Create the index
res = redis.ft(index_name).create_index(fields=schema, definition=definition)

print(res)

OK


Agora, vamos apenas garantir que a indexação foi tranquila... sem surpresas!

In [ ]:
info = redis.ft("idx:bikes_vss").info()
num_docs = info["num_docs"]
indexing_failures = info["hash_indexing_failures"]

print("Documentos indexados: {0}".format(num_docs))

print("Falhas de Indexação: {0}".format(indexing_failures))

Documentos indexados: 11
Falhas de Indexação: 0


# Segunda Parte - Usando o Redis como um Vector DB pra valer

Vou continuar no passo 7, pra não confundir vocês. Agora, vamos começar a brincar com os dados que estão lá no Redis.

Acredito que o caminho mais racional e simples será fazer o embedding das queries que clientes fariam normalmente.

Na minha humilde opinião, é o primeiro passo pra usar o Redis numa arquitetura RAG: vetores.

## 7 - Carregando algumas queries no Python

In [ ]:
queries = [
           "Bicicleta para crianças pequenas",
           "Melhores bicicletas de montanha para crianças",
           "Bicicleta de montanha barata para crianças",
           "Bicicleta de montanha específica para mulheres",
           "Bicicleta de estrada para iniciantes",
           "Bicicleta de comutação para pessoas com mais de 60 anos",
           "Bicicleta de comutação confortável",
           "Boa bicicleta para estudantes universitários",
           "Bicicleta de montanha para iniciantes",
           "Bicicleta vintage",
           "Bicicleta confortável para a cidade"
           ]


Vamos fazer o embedding de cada uma dessas queries... jogo rápido!

In [ ]:
encoded_queries = embedder.encode(queries)
len(encoded_queries)

print(encoded_queries)

[[ 0.02001647  0.737069   -0.16359873 ...  0.22224252  0.13291176
   0.1805346 ]
 [-0.11804421  0.6186317   0.02967137 ... -0.06789754  0.01700302
   0.14308521]
 [-0.07540765  0.7579044  -0.174123   ... -0.10637357 -0.18678775
   0.04395986]
 ...
 [-0.09652202  0.41005573 -0.2518025  ... -0.05905791 -0.03811348
  -0.12661776]
 [-0.14603637  0.73481745 -0.09919461 ... -0.27685368  0.26435688
   0.16712727]
 [ 0.51777124  0.49398148 -0.26587296 ...  0.13598055 -0.13339096
   0.14764398]]


## 8 - Busca K-nearest neighbors (KNN)

O algoritmo **KNN** calcula a distância entre o vetor de consulta e cada vetor no Redis com base na função de distância escolhida. Cosine, no nosso caso.

Em seguida, retorna os **top K** itens com as menores distâncias ao vetor de consulta. *Estes são os itens mais semanticamente similares.*

**Agora, construa uma consulta para fazer exatamente isso:**

In [ ]:
query = (
    Query('(*)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'id', 'brand', 'model', 'description')
     .dialect(2)
)

**Permita-me explicar aqui:**

- A expressão de filtro (*) significa todos. Em outras palavras, nenhum filtro foi aplicado. Você pode substituí-la por uma expressão que filtre por metadados adicionais. Inclusive GeoLocation.

- A parte **KNN** da consulta procura os 3 vizinhos mais próximos.
- O vetor de consulta deve ser passado como o parâmetro `query_vector`.
- A distância ao vetor de consulta é retornada como `vector_score`.
- Os resultados são classificados por este `vector_score`.
- Por fim, retorna os campos `vector_score, id, brand, model e description` para cada resultado.


## 9 - Como usar a query contra o Redis?

**Agora, você deve passar a consulta vetorizada como um array de bytes com o nome do parâmetro query_vector.**\
O código a seguir cria um array **NumPy** em Python a partir do vetor de consulta e o converte em uma representação compacta em nível de byte que pode ser passada como um parâmetro para a consulta:

```
redis.ft('idx:bikes_vss').search(
    query,
    {
      'query_vector': np.array(encoded_query, dtype=np.float32).tobytes()
    }
).docs
```

Com o template para a consulta pronto, você pode executar todas as consultas em um loop. Observe que o script calcula o vector_score para cada resultado como 1 - doc.vector_score. Como a distância cosseno é usada como a métrica, os itens com a menor distância estão mais próximos e, portanto, são mais similares à consulta.

Em seguida, faça um loop sobre os documentos correspondentes e crie uma lista de resultados que pode ser convertida em uma tabela Pandas para visualizar os resultados:

In [ ]:
def create_query_table(query, queries, encoded_queries, extra_params=None):
    """
    Creates a query table.
    """
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = (
            redis.ft("idx:bikes_vss")
            .search(
                query,
                {"query_vector": np.array(encoded_query, dtype=np.float32).tobytes()}
                | (extra_params if extra_params else {}),
            )
            .docs
        )
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append(
                {
                    "query": queries[i],
                    "score": vector_score,
                    "id": doc.id,
                    "brand": doc.brand,
                    "model": doc.model,
                    "description": doc.description,
                }
            )

    # Optional: convert the table to Markdown using Pandas
    queries_table = pd.DataFrame(results_list)
    queries_table.sort_values(
        by=["query", "score"], ascending=[True, False], inplace=True
    )
    queries_table["query"] = queries_table.groupby("query")["query"].transform(
        lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
    )
    queries_table["description"] = queries_table["description"].apply(
        lambda x: (x[:497] + "...") if len(x) > 500 else x
    )
    return queries_table.to_markdown(index=False)

Os resultados da consulta mostram as três melhores correspondências (nosso parâmetro K) de cada consulta individual, juntamente com o id, a marca e o modelo da bicicleta para cada consulta.

Por exemplo, para a consulta "Melhores bicicletas de montanha para crianças", a maior pontuação de similaridade (>0.60) e, portanto, a correspondência mais próxima foi a bicicleta da marca 'Nord' modelo 'Chook air 5', descrita como:



> A Chook Air 5 oferece às crianças a partir de seis anos uma bicicleta de montanha durável e superleve para sua primeira experiência em trilhas e cruzeiros fáceis por florestas e campos. O tubo superior mais baixo facilita montar e desmontar em qualquer situação, proporcionando mais segurança para seus filhos nas trilhas. A Chook Air 5 é a introdução perfeita ao mountain biking.

Pela descrição, esta bicicleta é uma excelente escolha para crianças mais novas, e os embeddings capturaram com precisão a semântica da descrição.



In [ ]:
query = (
    Query("(*)=>[KNN 3 @vector $query_vector AS vector_score]")
    .sort_by("vector_score")
    .return_fields("vector_score", "id", "brand", "model", "description")
    .dialect(2)
)

table = create_query_table(query, queries, encoded_queries)
print(table)

| query                                                   |   score | id        | brand      | model         | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:--------------------------------------------------------|--------:|:----------|:-----------|:--------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------